In [6]:
class Twisted_Edwards:
    # a * x**2 + y**2 = 1 + d * x**2 * y**2
    # In this challenge: x ** 2 + y ** 2 = 1 + 3617 * x ** 2 * y ** 2
    a, d = 1, 3617

class Montgomery:
    # B * y**2 = x**3 + A * x**2 + x
    pass

def isomorphism(T):
    # Twisted_Edwards to Weierstrass
    u, v = T
    
    # Twisted_Edwards to Montgomery
    # Ref: https://www-fourier.univ-grenoble-alpes.fr/mphell/doc-v5/conversion_weierstrass_edwards.html
    x = (1 + v) * pow(1 - v, -1, P) % P
    y = (1 + v) * pow((1 - v)*u, -1, P) % P
    
    # Montgomery to Weierstrass
    # Ref: https://math.stackexchange.com/questions/4180415/short-weierstrass-to-montgomery-curve-example
    X = x * pow(B, -1, P) + A * pow(3*B, -1, P)
    Y = y * pow(B, -1, P)
    
    return E(X, Y)

In [7]:
P    = 2 ** 414 - 17
a, d = 1, 3617

# Twisted_Edwards to Montgomery
A = 2 * (a + d) * pow(a - d, -1, P) % P
B = 4 * pow(a - d, -1, P) % P

# Montgomery to Weierstrass
a = (3 - A**2) * pow(3*B**2, -1, P) % P
b = (2*A**3 - 9*A) * pow(27*B**3, -1, P) % P
E = EllipticCurve(GF(P), [a, b])

# Twisted_Edwards Base
Gt = (17319886477121189177719202498822615443556957307604340815256226171904769976866975908866528699294134494857887698432266169206165, 34)

# Convert to Weierstrass Base
Gw = isomorphism(Gt)

# Convert enc to Weierstrass
Ct = (29389900956614406804195679733048238721927197300216785144586024378999988819550861039522005309555174206184334563744077143526515, 35393890305755889860162885313105764163711685229222879079392595581125504924571957049674311023316022028491019878405896203357959)
Cw = isomorphism(Ct)
hint = 323811241263249292936728039512527915123919581362694022248295847200852329370976362254362732891461683020125008591836401372097

In [8]:
Cw -= hint*Gw
G1, G2, G3 = 2**313*Gw, 2**196*Gw, 2**13*Gw
mask =   (1 << 412) - 1
mask ^^= ((1 << 22) -1) << 313
mask ^^= ((1 << 22) -1) << 13
mask ^^= 1 << 196

In [9]:
from tqdm import *

# Cw = (hint + x1*2**313 + x2*2**196 + x3*2**13)*Gw 

left = dict()
for x1 in tqdm(range(2**22)):
    left[Cw - x1*G1] = x1

save("log.sobj")
print("done!")

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4194304/4194304 [37:56<00:00, 1842.23it/s]


TypeError: save() takes at least 2 positional arguments (1 given)

In [10]:
left[Cw]

0

In [11]:
for x3 in tqdm(range(2**22)):
    for x2 in range(2):
        ok = x2*G2 + x3*G3
        if ok in left:
            print(left[ok], x2, x3)

 14%|█████████████████                                                                                                           | 577686/4194304 [09:16<1:03:12, 953.53it/s]

1927017 0 577508


 20%|█████████████████████████▌                                                                                                   | 859197/4194304 [14:16<55:23, 1003.45it/s]


KeyboardInterrupt: 

In [12]:
x1, x2, x3 = 1927017, 0 ,577508
order = 2 ** 411 - 33364140863755142520810177694098385178984727200411208589594759
C = hint + x1*2**313 + x2*2**196 + x3*2**13
m = pow(C, -1, order)

from Crypto.Util.number import *
print(long_to_bytes(m))

b'maple{th3_3dw4rd_cu12ve_and_kn0wn_bits_dlp}'
